In [139]:
#!/usr/bin/python

# General imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import pprint
import seaborn as sns
import sys

# Udacity module imports
sys.path.append("../tools/")
from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data
from tester import test_classifier

# Scikit-Learn imports
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB


# Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

In [140]:
# Task 1: Select what features you'll use.
# features_list is a list of strings, each of which is a feature name.
# The first feature must be "poi".

feature_list = [
    'poi',  
    'bonus',
    'deferral_payments',
    'deferred_income',
    'director_fees',
    #'email_address',
    'exercised_stock_options',
    'expenses',
    'from_messages',
    'from_poi_to_this_person',
    'from_this_person_to_poi',
    'loan_advances',
    'long_term_incentive',
    'other',
    'restricted_stock',
    'restricted_stock_deferred',
    'salary',
    'shared_receipt_with_poi',
    'to_messages',
    'total_payments',
    'total_stock_value'
]  



In [141]:
# Task 2: Remove outliers
data_dict.pop('TOTAL')

{'bonus': 97343619,
 'deferral_payments': 32083396,
 'deferred_income': -27992891,
 'director_fees': 1398517,
 'email_address': 'NaN',
 'exercised_stock_options': 311764000,
 'expenses': 5235198,
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 83925000,
 'long_term_incentive': 48521928,
 'other': 42667589,
 'poi': False,
 'restricted_stock': 130322299,
 'restricted_stock_deferred': -7576788,
 'salary': 26704229,
 'shared_receipt_with_poi': 'NaN',
 'to_messages': 'NaN',
 'total_payments': 309886585,
 'total_stock_value': 434509511}

In [142]:
# Task 3: Create new feature(s)



In [143]:
# Import data into dataframe for EDA
df = pd.DataFrame.from_records(list(data_dict.values()))
employees = pd.Series(list(data_dict.keys()))
df.set_index(employees, inplace=True)
df.replace('NaN', 0, inplace=True)

In [144]:
# EDA

#print('Shape: {}'.format(df.shape))
#print(df.describe())
#sns.boxplot(x='poi', y='bonus', data=df)
'''
g = sns.PairGrid(data=df, hue='poi')
g = g.map_diag(plt.hist)
g = g.map_offdiag(plt.scatter)
plt.show()
'''

"\ng = sns.PairGrid(data=df, hue='poi')\ng = g.map_diag(plt.hist)\ng = g.map_offdiag(plt.scatter)\nplt.show()\n"

In [145]:
# Extract features and labels from dataset for local testing
dataset = featureFormat(data_dict, feature_list, sort_keys=True)
labels, features = targetFeatureSplit(dataset)

# Split data into training and test sets
X_train, X_test, y_train, y_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

In [150]:
scaler = MinMaxScaler()
feature_selection = SelectKBest(score_func=chi2, k=5)
classifier = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, random_state=42)

clf = Pipeline([
    ('scaler', scaler),
    ('feature_selection', feature_selection),
    ('classifier', classifier)
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['non-poi', 'poi']))

             precision    recall  f1-score   support

    non-poi       0.91      0.82      0.86        39
        poi       0.22      0.40      0.29         5

avg / total       0.84      0.77      0.80        44



In [ ]:
scaler = MinMaxScaler()
feature_selection = SelectKBest(score_func=chi2, k=5)
classifier = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, random_state=42)

clf = Pipeline([
    ('scaler', scaler),
    ('feature_selection', feature_selection),
    ('classifier', classifier)
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['non-poi', 'poi']))

In [149]:
# Task 4: Try a varity of classifiers
# Please name your classifier clf for easy export below.
# Note that if you want to do PCA or other multi-stage operations,
# you'll need to use Pipelines. For more info:
# http://scikit-learn.org/stable/modules/pipeline.html



# Task 5: Tune your classifier to achieve better than .3 precision and recall
# using our testing script. Check the tester.py script in the final project
# folder for details on the evaluation method, especially the test_classifier
# function. Because of the small size of the dataset, the script uses
# stratified shuffle split cross validation. For more info:
# http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

#test_classifier(clf, data_dict, feature_list, folds = 1000)

# Task 6: Dump your classifier, dataset, and features_list so anyone can
# check your results. You do not need to change anything below, but make sure
# that the version of poi_id.py that you submit can be run on its own and
# generates the necessary .pkl files for validating your results.

# dump_classifier_and_data(clf, my_dataset, features_list)


if __name__ == '__main__':
    pass
    #test_multiple(classifier_types, features_train, features_test, labels_train,
    #labels_test)




Pipeline(steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('feature_selection', SelectKBest(k=5, score_func=<function chi2 at 0x115cf26e0>)), ('classifier', AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=...ndom_state=None, splitter='best'),
          learning_rate=1.0, n_estimators=100, random_state=42))])
	Accuracy: 0.81160	Precision: 0.30163	Recall: 0.31400	F1: 0.30769	F2: 0.31145
	Total predictions: 15000	True positives:  628	False positives: 1454	False negatives: 1372	True negatives: 11546

